## Multivariate Linear Regression

In [49]:
import numpy as np
import pandas as pd
import math

In [132]:
class LinearRegression:
        
    def hyp(self):
        return np.dot(self.X, self.theta)
        
    def get_cost(self):
        return sum((self.hyp() - self.y)**2) / (self.m * 2.0)
    
    def fscale(self):
        self.X= self.X / (self.X.max(axis= 0) - self.X.min(axis= 0))
        self.y= self.y / (max(self.y) - min(self.y))
        
    def computeStandard(self):
        from sklearn.linear_model import LinearRegression
        self.linreg= LinearRegression()
        self.linreg.fit(self.X, self.y)

    def preprocess(self, X, y, alpha):
        (self.m, self.n)= X.shape
        self.X= X.values
        self.y= y.values
        self.fscale()
        self.computeStandard()
        self.n+=1
        self.X= np.insert(self.X, 0, 1, axis= 1)
        self.y= self.y[None].transpose()
        self.alpha= alpha
        self.theta= (np.ones(self.n)*10)[None].transpose()
        
    def fit(self, X, y, alpha= 0.1):
        self.preprocess(X, y, alpha)
        pc= math.inf
        change= 1
        iteration=0
        while(change>0.000000000001):
            der = (self.hyp() - self.y).transpose()
            der = np.dot(der , self.X) * self.alpha / self.m
            self.theta= self.theta - der.transpose()

            self.cost= self.get_cost()
            change= pc - self.cost
            pc= self.cost
            iteration += 1
            if(iteration % 500 == 0):
                print("{}. Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
        print('+'*70, "\n{}. Final Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
        print("sklearn: theta: ",self.linreg.intercept_, self.linreg.coef_,"\n")
        
            
    def predict(self, X):
        return None
    
    

### Demo Dataset

In [133]:
np.random.seed(42)
a= np.random.randint(1, 25, 5)
b= np.random.randint(100, 1000, 5)
c= 11 + 2 * a + 5 * b
data= pd.DataFrame({"TV":a, "Radio":b, "Sales":c})
X= data[['TV','Radio']]
y= data['Sales']

In [134]:
linreg= LinearRegression()
linreg.fit(X, y, alpha= 0.3)

500. Cost: [6.98846339e-06], theta: [[-0.01530652  0.0193483   1.01781089]]
1000. Cost: [5.08362949e-09], theta: [[0.00275946 0.00801999 1.00797853]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
1298. Final Cost: [6.85175838e-11], theta: [[0.00320209 0.00774244 1.00773763]]
sklearn: theta:  0.003260225251926596 [0.00770599 1.00770599] 



### Advertising Dataset

In [135]:
data= pd.read_csv('Files/Advertising.csv', index_col= 0)
feature_cols= ['TV', 'Radio', 'Newspaper']
target_col= 'Sales'
X= data[feature_cols]
y= data[target_col]

In [137]:
linreg= LinearRegression()
linreg.fit(X, y, alpha= 0.7)

500. Cost: [0.0021577], theta: [[ 0.11568268  0.53278042  0.36810195 -0.00447804]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
572. Final Cost: [0.0021577], theta: [[ 0.11569944  0.5327799   0.36814158 -0.00460679]]
sklearn: theta:  0.11570430588422892 [ 0.53277975  0.3681531  -0.00464421] 

